In [37]:
# Data manipulation
from sklearn.preprocessing import LabelEncoder
from scipy.stats import loguniform
import numpy as np
import pandas as pd
import os

# Data Visualazation
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme()
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# Machine learning
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
import catboost as cb
from catboost import CatBoostRegressor
import xgboost as xgb
from xgboost import XGBRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.neural_network import MLPRegressor

# Model Evaluations
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

# print module versions for reproducibility
print('CatBoost version {}'.format(cb.__version__))
print('NumPy version {}'.format(np.__version__))
print('Pandas version {}'.format(pd.__version__))
print('seaborn version {}'.format(sns.__version__))
print('sklearn version {}'.format(sk.__version__))
print('xgboost version {}'.format(xgb.__version__))
print('lightgbm version {}'.format(lgb.__version__))

# User Defined Functions

# a) BoxPlot:- To check Outliers
def Create_BoxPlotS(DataFrame, Features, Rows, Columns):
    fig=plt.figure(figsize=(20,20))
    for i, Feature in enumerate(Features[0:]):
        ax=fig.add_subplot(Rows,Columns,i+1)
        sns.boxplot(DataFrame[Feature],ax=ax)   
    fig.tight_layout()  
    plt.show()
    
# b) DistPlot:- To check Normality
def Create_DistPlots(DataFrame, Features, Rows, Columns):
    fig=plt.figure(figsize=(20,20))
    for i, feature in enumerate(Features[0:]):
        ax=fig.add_subplot(Rows,Columns,i+1)
        sns.distplot(DataFrame[feature],ax=ax)    
    fig.tight_layout()  
    plt.show()

# c) Univariate Analysis of Catagorical Variables
def Univariate_Cat_Features(DataFrame, Features, Rows, Columns):
    fig=plt.figure(figsize=(20,20))
    for i, feature in enumerate(Features[0:]):
        ax=fig.add_subplot(Rows,Columns,i+1)
        sns.countplot(DataFrame[feature],ax=ax)    
    fig.tight_layout()  
    plt.show()
    
# d) Scatter Plot:- Bivariate Analysis between Target Variable & Independent Variables
# Note:- Pass Target_Variable in Single Quote in this function
def Create_ScatterPlots(DataFrame, Features, Target_Variable, Rows, Columns):
    fig=plt.figure(figsize=(20,20))
    for i, feature in enumerate(Features[0:]):
        ax=fig.add_subplot(Rows,Columns,i+1)
        sns.scatterplot(DataFrame[feature],DataFrame[Target_Variable],ax=ax)  
    fig.tight_layout()  
    plt.show()

# e) Joint Plot:- Bivariate Analysis between Target Variable & Independent Variables
# Note:- Pass Target_Variable in Single Quote in this function
def Create_JointPlots(DataFrame, Features, Target_Variable):
    fig=plt.figure(figsize=(20,20))
    for i, feature in enumerate(Features[0:]):
        sns.jointplot(y= Target_Variable, x= DataFrame[feature], data = DataFrame,kind='reg')
    fig.tight_layout()  
    plt.show()

# f) Check Feature wise Outliers   
def Check_Outliers(DataFrame, Column_List):
    for column in Column_List:    
        q1 = DataFrame[column].quantile(0.25)    # First Quartile
        q3 = DataFrame[column].quantile(0.75)    # Third Quartile
        IQR = q3 - q1                            # Inter Quartile Range
        llimit = q1 - 1.5*IQR                    # Lower Limit
        ulimit = q3 + 1.5*IQR                    # Upper Limit
        outliers = DataFrame[(DataFrame[column] < llimit) | (DataFrame[column] > ulimit)]
        print('\nNumber of outliers in "' + column + '" :' + str(len(outliers)))
        print('\nLower Limit in "' + column + '" :',llimit)
        print('\nUpper Limit in "' + column + '" :',ulimit)
        print('\nInter Quartile Range of "' + column + '" :',IQR,"\n")

# g) Label Encoding   
def Label_Encoder(DataFrame):
    le = LabelEncoder()
    for col in DataFrame.columns:
        if(DataFrame[col].dtype == 'object'):
            DataFrame.loc[:,col] = le.fit_transform(DataFrame.loc[:,col])
            
            
Ind_train = pd.read_csv (r'C:\Users\1014070\Favorites\AB\MH\Sustainability-June23\Data\India_Train-Processed.csv')
Ind_test = pd.read_csv (r'C:\Users\1014070\Favorites\AB\MH\Sustainability-June23\Data\India_Test-Processed.csv')
Label_Encoder(Ind_train)
Label_Encoder(Ind_test)

CatBoost version 1.2
NumPy version 1.21.5
Pandas version 1.4.2
seaborn version 0.11.2
sklearn version 1.0.2
xgboost version 1.7.6
lightgbm version 3.3.5


In [38]:
# print(Ind_train.columns.values)

# Ind_train = Ind_train.rename(columns={"State Name": "STATE", "Dist Name": "DISTRICT"
#                                       , "COTTON AREA (1000 ha)": "COTTON_AREA(1000Ha)"
#                                       ,"COTTON PRODUCTION (1000 tons)": "COTTON_PRODUCTION(1000Tons)"
#                                      ,"JANUARY PERCIPITATION (Millimeters)": "JAN_Actual_RF(mm)"
#                                       ,"FEBRUARY PERCIPITATION (Millimeters)": "FEB_Actual_RF(mm)"
#                                      ,"MARCH PERCIPITATION (Millimeters)": "MAR_Actual_RF(mm)"
#                                       ,"APRIL PERCIPITATION (Millimeters)": "APR_Actual_RF(mm)"
#                                      ,"MAY PERCIPITATION (Millimeters)": "MAY_Actual_RF(mm)"
#                                      ,"JUNE PERCIPITATION (Millimeters)": "STATE","State Name": "STATE"
#                                      ,"State Name": "STATE","State Name": "STATE"
#                                      ,"State Name": "STATE","State Name": "STATE"
#                                      ,"State Name": "STATE","State Name": "STATE"},
#     inplace=True,
# )



In [39]:
from sklearn.metrics import mean_squared_error

# Prepare the India training data for modeling
india_features = Ind_train.drop(["COTTON YIELD (Kg per ha)"], axis=1)
india_target = Ind_train["COTTON YIELD (Kg per ha)"]
Ind_test = Ind_test.drop(['COTTON YIELD (Kg per ha)'],axis=1)

# Split the data into training set and validation set
X_train, X_val, y_train, y_val = train_test_split(
    india_features, india_target, test_size=0.15, random_state=0
)

In [40]:
model_IND1 = LinearRegression() #random_state not required  #(XXXX)
model_IND1.fit(X_train,y_train) 
print('Accuracy of Linear Regression on training set: {:.2f}'
     .format(model_IND1.score(X_train, y_train)))

print('-------------'*9)

# # Predict the validation set results and calculate MSE
y_pred_val = model_IND1.predict(X_val)
mse_india = mean_squared_error(y_val, y_pred_val)
print(f"India Validation MSE: {mse_india}")

# 1st Version of Train
# Accuracy of Linear Regression on training set: 0.33
# Accuracy of Linear Regression on test set: 0.35
# 2nd Version of Train

Accuracy of Linear Regression on training set: 0.33
---------------------------------------------------------------------------------------------------------------------
India Validation MSE: 13350030.465085685


In [41]:
model_IND2 = DecisionTreeRegressor(random_state=0) #(XXXX)
model_IND2.fit(X_train,y_train) 
print('Accuracy of Decision Tree Regressor on training set: {:.2f}'
     .format(model_IND2.score(X_train, y_train)))

print('-------------'*9)

# # Predict the validation set results and calculate MSE
y_pred_val2 = model_IND2.predict(X_val)
mse_india = mean_squared_error(y_val, y_pred_val2)
print(f"India Validation MSE: {mse_india}")
# 1st Version of Train
# Accuracy of Decision Tree Regressor on training set: 1.00
# Accuracy of Decision Tree Regressor on test set: 0.60
# 2nd Version of Train

Accuracy of Decision Tree Regressor on training set: 1.00
---------------------------------------------------------------------------------------------------------------------
India Validation MSE: 7109124.465883941


In [42]:
model_IND3 = RandomForestRegressor(random_state=0) #(YYYYY)
model_IND3.fit(X_train,y_train) 
print('Accuracy of Random Forest Regressor on training set: {:.2f}'
     .format(model_IND3.score(X_train, y_train)))

print('-------------'*9)

# # Predict the validation set results and calculate MSE
y_pred_val3 = model_IND3.predict(X_val)
mse_india = mean_squared_error(y_val, y_pred_val3)
print(f"India Validation MSE: {mse_india}")
# 1st Version of Train
# Accuracy of Random Forest Regressor on training set: 0.95
# Accuracy of Random Forest Regressor on test set: 0.85
# 2nd Version of Train

Accuracy of Random Forest Regressor on training set: 0.95
---------------------------------------------------------------------------------------------------------------------
India Validation MSE: 3121070.067022587


In [43]:
model_IND4 = CatBoostRegressor(random_state=0,logging_level='Silent') #(YYYYY)
model_IND4.fit(X_train,y_train) 
print('Accuracy of Cat Boost Regressor on training set: {:.2f}'
     .format(model_IND4.score(X_train, y_train)))

print('-------------'*9)

# # Predict the validation set results and calculate MSE
y_pred_val4 = model_IND4.predict(X_val)
mse_india = mean_squared_error(y_val, y_pred_val4)
print(f"India Validation MSE: {mse_india}")
# 1st Version of Train
# Accuracy of Cat Boost Regressor on training set: 0.99
# Accuracy of Cat Boost Regressor on test set: 0.87
# 2nd Version of Train

Accuracy of Cat Boost Regressor on training set: 0.99
---------------------------------------------------------------------------------------------------------------------
India Validation MSE: 3160173.0597800403


In [44]:
model_IND5 = XGBRegressor(random_state=0) ###(YYYY)
model_IND5.fit(X_train,y_train) 
print('Accuracy of XGB Regressor on training set: {:.2f}'
     .format(model_IND5.score(X_train, y_train)))

print('-------------'*9)

# # Predict the validation set results and calculate MSE
y_pred_val5 = model_IND5.predict(X_val)
mse_india = mean_squared_error(y_val, y_pred_val5)
print(f"India Validation MSE: {mse_india}")
# 1st Version of Train
# Accuracy of XGB Regressor on training set: 1.00
# Accuracy of XGB Regressor on test set: 0.84
# 2nd Version of Train

Accuracy of XGB Regressor on training set: 1.00
---------------------------------------------------------------------------------------------------------------------
India Validation MSE: 3210377.625395154


In [45]:
model_IND6 = LGBMRegressor(random_state=0) ###(yyyy)
model_IND6.fit(X_train,y_train) 
print('Accuracy of LGBM Regressor on training set: {:.2f}'
     .format(model_IND6.score(X_train, y_train)))
print('-------------'*9)

# # Predict the validation set results and calculate MSE
y_pred_val6 = model_IND6.predict(X_val)
mse_india = mean_squared_error(y_val, y_pred_val6)
print(f"India Validation MSE: {mse_india}")

# 1st Version of Train
# Accuracy of LGBM Regressor on training set: 0.96
# Accuracy of LGBM Regressor on test set: 0.85
# 2nd Version of Train

Accuracy of LGBM Regressor on training set: 0.96
---------------------------------------------------------------------------------------------------------------------
India Validation MSE: 3171813.3588705678


In [46]:
model_IND7 = GradientBoostingRegressor(random_state=0) ###(YYYY)
model_IND7.fit(X_train,y_train) 
print('Accuracy of Gradient Boosting Regressor on training set: {:.2f}'
     .format(model_IND7.score(X_train, y_train)))

print('-------------'*9)

# # Predict the validation set results and calculate MSE
y_pred_val7 = model_IND7.predict(X_val)
mse_india = mean_squared_error(y_val, y_pred_val7)
print(f"India Validation MSE: {mse_india}")

# 1st Version of Train
# Accuracy of Gradient Boosting Regressor on training set: 0.92
# Accuracy of Gradient Boosting Regressor on test set: 0.82
# 2nd Version of Train

Accuracy of Gradient Boosting Regressor on training set: 0.92
---------------------------------------------------------------------------------------------------------------------
India Validation MSE: 3808972.446719119


In [47]:
model_IND8 = Lasso(random_state=0) ###(nnnn)
model_IND8.fit(X_train,y_train) 
print('Accuracy of Lasso Regression on training set: {:.2f}'
     .format(model_IND8.score(X_train, y_train)))

print('-------------'*9)

# # Predict the validation set results and calculate MSE
y_pred_val8 = model_IND8.predict(X_val)
mse_india = mean_squared_error(y_val, y_pred_val8)
print(f"India Validation MSE: {mse_india}")
# 1st Version of Train
# Accuracy of Lasso Regression on training set: 0.33
# Accuracy of Lasso Regression on test set: 0.35
# 2nd Version of Train

Accuracy of Lasso Regression on training set: 0.33
---------------------------------------------------------------------------------------------------------------------
India Validation MSE: 13347688.13695243


In [48]:
model_IND9 = Ridge(random_state=0) ###(nnnn)
model_IND9.fit(X_train,y_train) 
print('Accuracy of Ridge Regressor on training set: {:.2f}'
     .format(model_IND9.score(X_train, y_train)))

print('-------------'*9)
# # Predict the validation set results and calculate MSE
y_pred_val9 = model_IND9.predict(X_val)
mse_india = mean_squared_error(y_val, y_pred_val9)
print(f"India Validation MSE: {mse_india}")

# 1st Version of Train
# Accuracy of Ridge Regressor on training set: 0.33
# Accuracy of Ridge Regressor on test set: 0.35
# 2nd Version of Train

Accuracy of Ridge Regressor on training set: 0.33
---------------------------------------------------------------------------------------------------------------------
India Validation MSE: 13350003.884934636


In [49]:
model_IND10 = HistGradientBoostingRegressor(random_state = 0) ###(yyyyy)
model_IND10.fit(X_train,y_train) 
print('Accuracy of Hist Gradient Boosting Regressor on training set: {:.2f}'
     .format(model_IND10.score(X_train, y_train)))

print('-------------'*9)
# # Predict the validation set results and calculate MSE
y_pred_val10 = model_IND10.predict(X_val)
mse_india = mean_squared_error(y_val, y_pred_val10)
print(f"India Validation MSE: {mse_india}")

# 1st Version of Train
# Accuracy of Hist Gradient Boosting Regressor on training set: 0.96
# Accuracy of Hist Gradient Boosting Regressor on test set: 0.84
# 2nd Version of Train

Accuracy of Hist Gradient Boosting Regressor on training set: 0.96
---------------------------------------------------------------------------------------------------------------------
India Validation MSE: 3290360.4131322773


In [50]:
model_IND11 = MLPRegressor(random_state=0) ###(nnnn)
model_IND11.fit(X_train,y_train) 
print('Accuracy of MLPRegressor() on training set: {:.2f}'
     .format(model_IND11.score(X_train, y_train)))

print('-------------'*9)
# # Predict the validation set results and calculate MSE
y_pred_val11 = model_IND11.predict(X_val)
mse_india = mean_squared_error(y_val, y_pred_val11)
print(f"India Validation MSE: {mse_india}")

# 1st Version of Train
# Accuracy of MLPRegressor() on training set: 0.56
# Accuracy of MLPRegressor() on test set: 0.58
# 2nd Version of Train

Accuracy of MLPRegressor() on training set: 0.54
---------------------------------------------------------------------------------------------------------------------
India Validation MSE: 9196354.395375738


In [51]:
# Import Test data for the prediction of the Target Variable
x = np.array(Ind_test)
y_pred_IND1 = model_IND1.predict(x)
y_pred_IND2 = model_IND2.predict(x)
y_pred_IND3 = model_IND3.predict(x)
y_pred_IND4 = model_IND4.predict(x)
y_pred_IND5 = model_IND5.predict(x)
y_pred_IND6 = model_IND6.predict(x)
y_pred_IND7 = model_IND7.predict(x)
y_pred_IND8 = model_IND8.predict(x)
y_pred_IND9 = model_IND9.predict(x)
y_pred_IND10 = model_IND10.predict(x)
y_pred_IND11 = model_IND11.predict(x)


In [52]:
submission_india=pd.DataFrame(y_pred_IND5,columns=['COTTON YIELD (Kg per ha)'])

#submission_india = submission_india.where(submission_india['COTTON YIELD (Kg per ha)']< 0, 0)

# USA

In [53]:
usa_train = pd.read_csv(r"C:\Users\1014070\Favorites\AB\MH\Sustainability-June23\Data\USA_train.csv")
usa_test = pd.read_csv(r"C:\Users\1014070\Favorites\AB\MH\Sustainability-June23\Data\USA_test.csv")

usa_train=usa_train.fillna("0")
usa_test=usa_test.fillna("0")

usa_train= usa_train.astype(float, errors='ignore')
usa_test= usa_test.astype(float, errors='ignore')

In [54]:
usa_train=usa_train.drop(['State','Planted (1000 Acres)',"Harvested (1000 Acres)"], axis=1)
usa_test=usa_test.drop(['State','Planted (1000 Acres)',"Harvested (1000 Acres)"], axis=1)
# Label_Encoder(usa_train)
# Label_Encoder(usa_test)

In [55]:
from sklearn.metrics import mean_squared_error

# Prepare the India training data for modeling
usa_features = usa_train.drop(["Yield (Pounds/ Harvested Area)"], axis=1)
usa_target = usa_train["Yield (Pounds/ Harvested Area)"]

# Split the data into training set and validation set
X_train, X_val, y_train, y_val = train_test_split(
    usa_features, usa_target, test_size=0.15, random_state=0
)

In [56]:
USAmodel1 = LinearRegression() #random_state not required 
USAmodel1.fit(X_train,y_train) 
print('Accuracy of Linear Regression on training set: {:.2f}'
     .format(USAmodel1.score(X_train, y_train)))

print('-------------'*9)

# # Predict the validation set results and calculate MSE
y_pred_val = USAmodel1.predict(X_val)
mse_india = mean_squared_error(y_val, y_pred_val)
print(f"India Validation MSE: {mse_india}")

# 1st Version of Train
# Accuracy of Linear Regression on training set: 0.33
# Accuracy of Linear Regression on test set: 0.35
# 2nd Version of Train

Accuracy of Linear Regression on training set: 0.27
---------------------------------------------------------------------------------------------------------------------
India Validation MSE: 76046.41220431059


In [57]:
USAmodel2 = DecisionTreeRegressor(random_state=0)
USAmodel2.fit(X_train,y_train) 
print('Accuracy of Decision Tree Regressor on training set: {:.2f}'
     .format(USAmodel2.score(X_train, y_train)))

print('-------------'*9)

# # Predict the validation set results and calculate MSE
y_pred_val2 = USAmodel2.predict(X_val)
mse_india = mean_squared_error(y_val, y_pred_val2)
print(f"India Validation MSE: {mse_india}")
# 1st Version of Train
# Accuracy of Decision Tree Regressor on training set: 1.00
# Accuracy of Decision Tree Regressor on test set: 0.60
# 2nd Version of Train

Accuracy of Decision Tree Regressor on training set: 1.00
---------------------------------------------------------------------------------------------------------------------
India Validation MSE: 75083.07142857143


In [58]:
USAmodel3 = RandomForestRegressor(random_state=0)
USAmodel3.fit(X_train,y_train) 
print('Accuracy of Random Forest Regressor on training set: {:.2f}'
     .format(USAmodel3.score(X_train, y_train)))

print('-------------'*9)

# # Predict the validation set results and calculate MSE
y_pred_val3 = USAmodel3.predict(X_val)
mse_india = mean_squared_error(y_val, y_pred_val3)
print(f"India Validation MSE: {mse_india}")
# 1st Version of Train
# Accuracy of Random Forest Regressor on training set: 0.95
# Accuracy of Random Forest Regressor on test set: 0.85
# 2nd Version of Train

Accuracy of Random Forest Regressor on training set: 0.94
---------------------------------------------------------------------------------------------------------------------
India Validation MSE: 38187.164559523815


In [59]:
USAmodel4 = CatBoostRegressor(random_state=0,logging_level='Silent')
USAmodel4.fit(X_train,y_train) 
print('Accuracy of Cat Boost Regressor on training set: {:.2f}'
     .format(USAmodel4.score(X_train, y_train)))

print('-------------'*9)

# # Predict the validation set results and calculate MSE
y_pred_val4 = USAmodel4.predict(X_val)
mse_india = mean_squared_error(y_val, y_pred_val4)
print(f"India Validation MSE: {mse_india}")
# 1st Version of Train
# Accuracy of Cat Boost Regressor on training set: 0.99
# Accuracy of Cat Boost Regressor on test set: 0.87
# 2nd Version of Train

Accuracy of Cat Boost Regressor on training set: 1.00
---------------------------------------------------------------------------------------------------------------------
India Validation MSE: 39780.25793132627


In [60]:
USAmodel5 = XGBRegressor(random_state=0)
USAmodel5.fit(X_train,y_train) 
print('Accuracy of XGB Regressor on training set: {:.2f}'
     .format(USAmodel5.score(X_train, y_train)))

print('-------------'*9)

# # Predict the validation set results and calculate MSE
y_pred_val5 = USAmodel5.predict(X_val)
mse_india = mean_squared_error(y_val, y_pred_val5)
print(f"India Validation MSE: {mse_india}")
# 1st Version of Train
# Accuracy of XGB Regressor on training set: 1.00
# Accuracy of XGB Regressor on test set: 0.84
# 2nd Version of Train

Accuracy of XGB Regressor on training set: 1.00
---------------------------------------------------------------------------------------------------------------------
India Validation MSE: 42712.671724585816


In [61]:
USAmodel6 = LGBMRegressor(random_state=0)
USAmodel6.fit(X_train,y_train) 
print('Accuracy of LGBM Regressor on training set: {:.2f}'
     .format(USAmodel6.score(X_train, y_train)))
print('-------------'*9)

# # Predict the validation set results and calculate MSE
y_pred_val6 = USAmodel6.predict(X_val)
mse_india = mean_squared_error(y_val, y_pred_val6)
print(f"India Validation MSE: {mse_india}")

# 1st Version of Train
# Accuracy of LGBM Regressor on training set: 0.96
# Accuracy of LGBM Regressor on test set: 0.85
# 2nd Version of Train

Accuracy of LGBM Regressor on training set: 0.99
---------------------------------------------------------------------------------------------------------------------
India Validation MSE: 42396.82458163869


In [62]:
USAmodel7 = GradientBoostingRegressor(random_state=0)
USAmodel7.fit(X_train,y_train) 
print('Accuracy of Gradient Boosting Regressor on training set: {:.2f}'
     .format(USAmodel7.score(X_train, y_train)))

print('-------------'*9)

# # Predict the validation set results and calculate MSE
y_pred_val7 = USAmodel7.predict(X_val)
mse_india = mean_squared_error(y_val, y_pred_val7)
print(f"India Validation MSE: {mse_india}")

# 1st Version of Train
# Accuracy of Gradient Boosting Regressor on training set: 0.92
# Accuracy of Gradient Boosting Regressor on test set: 0.82
# 2nd Version of Train

Accuracy of Gradient Boosting Regressor on training set: 0.90
---------------------------------------------------------------------------------------------------------------------
India Validation MSE: 45099.047584151114


In [63]:
USAmodel8 = Lasso(random_state=0)
USAmodel8.fit(X_train,y_train) 
print('Accuracy of Lasso Regression on training set: {:.2f}'
     .format(USAmodel8.score(X_train, y_train)))

print('-------------'*9)

# # Predict the validation set results and calculate MSE
y_pred_val8 = USAmodel8.predict(X_val)
mse_india = mean_squared_error(y_val, y_pred_val8)
print(f"India Validation MSE: {mse_india}")
# 1st Version of Train
# Accuracy of Lasso Regression on training set: 0.33
# Accuracy of Lasso Regression on test set: 0.35
# 2nd Version of Train

Accuracy of Lasso Regression on training set: 0.27
---------------------------------------------------------------------------------------------------------------------
India Validation MSE: 75850.1313875543


In [64]:
USAmodel9 = Ridge(random_state=0)
USAmodel9.fit(X_train,y_train) 
print('Accuracy of Ridge Regressor on training set: {:.2f}'
     .format(USAmodel9.score(X_train, y_train)))

print('-------------'*9)
# # Predict the validation set results and calculate MSE
y_pred_val9 = USAmodel9.predict(X_val)
mse_india = mean_squared_error(y_val, y_pred_val9)
print(f"India Validation MSE: {mse_india}")

# 1st Version of Train
# Accuracy of Ridge Regressor on training set: 0.33
# Accuracy of Ridge Regressor on test set: 0.35
# 2nd Version of Train

Accuracy of Ridge Regressor on training set: 0.27
---------------------------------------------------------------------------------------------------------------------
India Validation MSE: 76039.47443631775


In [65]:
USAmodel10 = HistGradientBoostingRegressor(random_state = 0)
USAmodel10.fit(X_train,y_train) 
print('Accuracy of Hist Gradient Boosting Regressor on training set: {:.2f}'
     .format(USAmodel10.score(X_train, y_train)))

print('-------------'*9)
# # Predict the validation set results and calculate MSE
y_pred_val10 = USAmodel10.predict(X_val)
mse_india = mean_squared_error(y_val, y_pred_val10)
print(f"India Validation MSE: {mse_india}")

# 1st Version of Train
# Accuracy of Hist Gradient Boosting Regressor on training set: 0.96
# Accuracy of Hist Gradient Boosting Regressor on test set: 0.84
# 2nd Version of Train

Accuracy of Hist Gradient Boosting Regressor on training set: 0.99
---------------------------------------------------------------------------------------------------------------------
India Validation MSE: 41046.55705275732


In [66]:
USAmodel11 = MLPRegressor(random_state=0)
USAmodel11.fit(X_train,y_train) 
print('Accuracy of MLPRegressor() on training set: {:.2f}'
     .format(USAmodel11.score(X_train, y_train)))

print('-------------'*9)
# # Predict the validation set results and calculate MSE
y_pred_val11 = USAmodel11.predict(X_val)
mse_india = mean_squared_error(y_val, y_pred_val11)
print(f"India Validation MSE: {mse_india}")

# 1st Version of Train
# Accuracy of MLPRegressor() on training set: 0.56
# Accuracy of MLPRegressor() on test set: 0.58
# 2nd Version of Train

Accuracy of MLPRegressor() on training set: 0.28
---------------------------------------------------------------------------------------------------------------------
India Validation MSE: 69290.32299572711


In [67]:

usa_test = usa_test.drop(['Yield (Pounds/ Harvested Area)'],axis=1)

# Import Test data for the prediction of the Target Variable
x = np.array(usa_test)
y_pred_USA1 = USAmodel1.predict(x)
y_pred_USA2 = USAmodel2.predict(x)
y_pred_USA3 = USAmodel3.predict(x)
y_pred_USA4 = USAmodel4.predict(x)
y_pred_USA5 = USAmodel5.predict(x)
y_pred_USA6 = USAmodel6.predict(x)
y_pred_USA7 = USAmodel7.predict(x)
y_pred_USA8 = USAmodel8.predict(x)
y_pred_USA9 = USAmodel9.predict(x)
y_pred_USA10 = USAmodel10.predict(x)
y_pred_USA11 = USAmodel11.predict(x)

In [72]:
submission_usa=pd.DataFrame(y_pred_USA3,columns=['Yield (Pounds/ Harvested Area)'])

In [73]:
commodity = pd.read_csv(r"C:\Users\1014070\Favorites\AB\MH\Sustainability-June23\Data\USA_Commodity_prices.csv")
commodity['Year']=pd.to_numeric(commodity['Year'])
submission_commodity=commodity[(commodity['Year']>=2003)&(commodity['Year']<=2022)]
submission_commodity=submission_commodity["Cotton_Price[Dollar/ton]"]

In [74]:
#submission = pd.concat([submission_india,submission_usa,submission_commodity],axis=1)

submission = pd.concat([submission_usa,submission_india,submission_commodity],axis=1) #14148922.62608

submission = submission.fillna(0)
submission.to_csv(r"C:\Users\1014070\Favorites\AB\MH\Sustainability-June23\Data\M3.csv",index=False)

In [75]:
print("Yes!")

Yes!
